<a href="https://colab.research.google.com/github/acrching/WhisperLive/blob/main/whisper%20streaming%20x%20chatgpt%20adpatation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install faster-whisper
!pip install streamlink
!pip install ffmpeg-python

In [7]:
import os
import time
import numpy as np
import queue
import threading
import ffmpeg
import streamlink
from faster_whisper import WhisperModel

AUDIO_BUFFER_SIZE = 30  # Buffer size in seconds

class WhisperOnline:
    def __init__(self, model_size, hls_url):
        self.audio_buffer = queue.Queue(maxsize=AUDIO_BUFFER_SIZE * 16000)
        self.model = WhisperModel(model_size)
        self.hls_url = hls_url

    def download_audio(self):
        streams = streamlink.streams(self.hls_url)
        stream_url = streams['best'].url

        process = (
            ffmpeg
            .input(stream_url)
            .output('pipe:', format='wav', acodec='pcm_s16le', ac=1, ar='16k')
            .run_async(pipe_stdout=True, pipe_stderr=True)
        )
        while True:
            in_bytes = process.stdout.read(1024)
            if not in_bytes:
                break
            self.audio_buffer.put(np.frombuffer(in_bytes, np.int16).astype(np.float32) / 32768.0)

    def transcribe_audio(self):
        while True:
            if not self.audio_buffer.empty():
                audio_chunk = []
                for _ in range(16000 * 10):
                    if not self.audio_buffer.empty():
                        audio_chunk.append(self.audio_buffer.get())
                    else:
                        break
                if len(audio_chunk) == 0:
                    break
                audio_chunk = np.concatenate(audio_chunk)
                segments, _ = self.model.transcribe(audio_chunk)
                for segment in segments:
                    print(segment.text)

    def run(self):
        download_thread = threading.Thread(target=self.download_audio)
        transcribe_thread = threading.Thread(target=self.transcribe_audio)

        download_thread.start()
        transcribe_thread.start()

        download_thread.join()
        transcribe_thread.join()

if __name__ == "__main__":
    model_size = 'large-v2'  # Model size for faster-whisper
    hls_url = 'https://dai.google.com/linear/hls/pa/event/Sid4xiTQTkCT1SLu6rjUSQ/stream/ff034e83-407c-4bbe-ab4b-0fd198258076:CBF2/master.m3u8'  # Replace with your HLS stream URL

    whisper_online = WhisperOnline(model_size, hls_url)
    whisper_online.run()

 you
 With CarGurus, you can buy or sell your car.
 your car, in person or online. If only you could do things your way all the time. Wouldn't
 that be nice? Get it with Gurus.
 you
 We in this together.
 Tulsa belongs to me now.
 Oh great, buckle up, here we go.
 I don't answer to any man.
 What happened to him?
 Yeah, he's taking a nap.
 ♪♪


KeyboardInterrupt: 